# AI智能风控(四）——风控策略的产品化全流程

本文作者：桔了个仔
知乎：https://www.zhihu.com/people/huangzhe



终于讲到第四节了。这节会分享一些少人知道的，教科书上不讲的东西，就是工程实践经验。当然，这个实践是我个人经验，我也相信并不是每个人都按照我的这个流程做，所以也欢迎大家在评论区分享自己做风控的经验。

如果你还没看过前三节，可以点击下方目录查看相应文章。不过这一节内容是独立于技术细节之上的，更偏重讲项目管理，甚至你不是做风控的，也可以看得懂。所以这节可以单独看，前面四节你可以先收藏晚点再看。

对于风控策略或模型产品化，一般会经过三个阶段：

## **一：概念验证(PoC)**

概念验证（PoC – Proof of Concept)是对你的方案的一个较短而不完整的实现，以证明其可行性，示范其原理。根据各公司管理不同，有的也叫PoV(Proof of Value)。或者换个熟悉的词，就是demo。如果你是第三方技术提供商，那一般都需要经过这个阶段。

一般PoC阶段包含这些事情：

- 设置成功条件。若达到这个成功条件，则认为PoC阶段成功，可以进入下个阶段，若不成功，如果允许
- 准备数据。按照方案里设计的数据schema提取数据。
- 调整模型，构建特征，达到预设的性能
- 展示结果，等待用户评估，验证，接受。

PoC目的是为了验证一些概念或理论。概念验证通常被认为是一个有里程碑意义的实现的原型 。如果确实不可行，尽快退出或者尽快修改方案，不然在产品化阶段才发现问题，那就麻烦了。

客户接受了之后，就可以进入下一个阶段，SIT。

## **二：系统集成测试(System integration testing, SIT)**

这个阶段有时候也叫Pilot(试运行）。事实上Pilot比SIT范围更大，但都可以理解成产品化前的试运行与模拟阶段，以观察系统与平台的适应性，发现潜在的bug。进入SIT段意味着马上就要产品化了，风控项目将变得更加复杂。这个阶段不再需要证明该解决方案的有效性，而是要将风控I项目集成到公司的基础架构中，并保证它在真实环境下能够很好地工作。

系统集成包括两方面：

- 你的技术方案内各子系统的集成。例如子系统A是模型自动训练模块，子系统B是模型评估模块，子系统A和B都能在PoC阶段单独运行得很好，但如果两个接连运行，有可能子系统B运行失败，原因可能是子系统A有内存泄露（还有什么可能性？读者也可以一起思考）。
- 你的技术方案与平台上其他方案是否兼容。例如你的技术方案需要用hive，但你公司的云平台上没安装，那么你就需要用看看这个公司能不能给你在他们平台上安装hive，如果不行，那么你的数据来源可能得改成他们目前用的数据库，例如MySQL等；又或者你做PoC时用的内存很大，但部署到客户环境发现客户的平台内存很有限。

系统集成如果都完成了，就要开始令人兴奋的产品化阶段了。

## **三：产品化阶段（PROD)**

到了产品化阶段，系统就要在线运行了，并且开始利用系统的预测结果来调整风控策略了。到了产品化阶段，原则上是不再调整方案，若小调整，得回到SIT环境做试验，若大调整，那就要重新做PoC，反正就是不能直接在PROD环境做调整。当然，直接用新方案代替旧方案可能有“翻车”风险，毕竟这和钱直接相关，不得不谨慎，所以一般分两个阶段运行。

- 阶段一：并行运行（新方案和旧方案并列运行，例如新方案检测20%的账户，旧方案检测另外80%的账户）
- 阶段二：取代原产品。并行运行一段时间（例如三个月），如果新方案一直比旧方案明显性能更好，则取代旧方案，负责100%的账户检测

关于各阶段的比较，我总结了一张表，如果小图不清晰可以打开原图。欢迎收藏。

![01](https://img.alicdn.com/imgextra/i4/O1CN01B25eod1baLN8l62zo_!!6000000003481-2-tps-2008-1438.png)

## **一些可能的Q&A（没有Q我，我自己来Q&A）**

**Q: 为什么要分PoC和SIT两个阶段呢？这两个阶段会有一些内容重叠，能不能合并呢？**

**A:** 理论上当然可以合并。有的公司就是这么做的。但是对于如果你是技术提供商，要把你服务卖给别人，那么PoC阶段就是一个双赢的。第一，减少试错成本。有的客户你接触后发现他们数据建设并不足以支撑你的方案，那么可以及早退出减少时间浪费，对彼此都是好的；第二，快速发现可能性。你在PoC阶段如果初步方案效果不理想，可以快速的实验其他方案；第三，PoC阶段的很多代码其实在SIT阶段和PROD阶段能继续使用的，只是实施方法不一样。

**Q: SIT和PROD为啥不合并？他们有啥不同**

**A:** 打个比方，SIT就是你上场之前的彩排。SIT和PROD的区别在于，SIT并不实际在使用，而在PROD里，你的模型预测会对金融机构的实际资产造成影响。而且SIT就像一个沙盒，就算你的系统有问题，也不会对线上其他正在运行的产品造成影响。

**Q: ”这好像不是我搞风控的流程啊，我的流程是【定义Y】-> 【数据清洗】→ 【特征构建】→ 【评估】→ 【上线】这样的啊。”**

**A:** 其实这只是看待的角度不一样，可能读者关注的是业务角度，但本文写的是从项目管理角度。如果把读者的角度和本文的角度对应起来，应该是这样的：

![02](https://img.alicdn.com/imgextra/i4/O1CN01LCnMZ128kli4Pic5Q_!!6000000007971-2-tps-1320-1504.png)

别看SIT和PROD虽然任务少，但是如果部署工作不是在自己常用的环境的话（例如客户的云平台），那么还是有很多琐事要处理的。

---

**桔了个仔：AI风控系列文章目录**

zhuanlan.zhihu.com


接下来还是会不定期更新的风控技术相关文章，我也会更新在目录里，但不再编号，每一篇都可以单独阅读，收藏目录，不会迷路。如果读者有不同的问题，欢迎留言， 如果我认为确实有必要，我会单独写文章来解答一下。

**希望这个系列大家喜欢。谢谢。**